# Optimize and group for tranfers (OPTIONAL)

Transfers can be done as individual `rclone copyto` statements but that is not efficient. It is the only approach when the name changes, though moves without renames work fine.

**WARNING**: This assumes all backups have been successful. If not, some of the final calls will fail. The files aren't lost *per se* but they are in an unaccounted for backup directory (i.e., the `back/<date>` is there but not the `<logs>/date`). Further forensics will be needed.

Algorithm:

We will have a `copyto` list, which needs to be done one-at-a-time. 

Then we will have a `copy` dict. The `<key> = <src_prefix>,<dst_prefix>` is the prefix and value is a list of `<files>` such that you would call:

    $ rclone copy backupdest:<src_prefix> restoredest:<dst_prefix> --files-from=file_with_<files>
    
 

In [ ]:
import json
import os
from pathlib import Path
from collections import defaultdict
from itertools import zip_longest

In [ ]:
with open('transfer_B_tracking.json') as fobj:
    transfers = json.load(fobj)
transfers

In [ ]:
copyto = []
copy = defaultdict(list)

for src,dst in transfers:
    src,dst = Path(src),Path(dst)
    
    if src.name != dst.name:
        copyto.append(tuple(map(str,(src,dst))))
        continue
    
    # Now we need to find the most common parts starting from the left
    for ii,(spart,dpart) in enumerate(zip_longest(src.parts[::-1],dst.parts[::-1],fillvalue=None)):
        if spart != dpart:
            src_prefix = '/'.join(src.parts[:-ii])
            dst_prefix = '/'.join(dst.parts[:-ii])
            file = '/'.join(src.parts[-ii:])
            break
    else:
        raise ValueError('Something is wrong')
            
    copy[src_prefix,dst_prefix].append(file)
    

In [ ]:
SAVE = False

In [ ]:
print(f"""\
Run the following replacing 'backupdest:' and 'restoredest:' (including potentially with directories after the colon).
Indented lines with '$' are commands to run. File listings are noted. Files will: {{{SAVE = }}}

Copyto:
""")

for src,dst in sorted(copyto):
    print(f"""    $ rclone copyto backupdest:{src} restoredest:{dst}""")

print("\nCopy:\n")

for ii,((src_prefix,dst_prefix),files) in enumerate(sorted(copy.items(),key=lambda i:(-len(i[1]),i))):
    print(f"""\n    $ rclone copy backupdest:{src_prefix} restoredest:{dst_prefix} --files-from files{ii:04d}.txt {"  # not saved" if not SAVE else ""}""")
    savetxt = '\n'.join(files)
    printtxt = '\n    '.join(files)
    print(f"""\n'files{ii:04d}.txt' ({"not saved" if not SAVE else "saved"}):\n    {printtxt}""")
    print('----')
    